In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense
from keras.models import Sequential

import glob, os, random

Using TensorFlow backend.


In [2]:
"""==========
   dataset 가져오기
   =========="""

base_path = 'dataset/training'

img_list = glob.glob(os.path.join(base_path, '*/*')) + glob.glob(os.path.join(base_path, '*/*/*'))

print(len(img_list))

4964


In [3]:
"""==========
   train, test dataset 만들기
   =========="""

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.1
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

# flow_from_directory() : 폴더 형테의 데이터 구조 가지고 옴
train_generator = train_datagen.flow_from_directory(
    base_path,
    target_size=(300, 300),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    seed=0
)

validation_generator = test_datagen.flow_from_directory(
    base_path,
    target_size=(300, 300),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    seed=0
)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

print(labels)

Found 4453 images belonging to 6 classes.
Found 491 images belonging to 6 classes.
{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}


In [4]:
"""==========
   모델 구성하기
   =========="""

model = Sequential([
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(pool_size=2),

    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),

    Flatten(),

    Dense(64, activation='relu'),

    Dense(6, activation='softmax')
])

In [5]:
"""==========
   모델 학습과정 설정하기
   =========="""

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 37, 37, 32)       

In [6]:
"""==========
   모델 학습하기_1
   =========="""

model.fit_generator(train_generator, steps_per_epoch=15, epochs=5, validation_data=validation_generator,validation_steps=5)

Epoch 1/5
15/15 [==============================] - 53s 4s/step - loss: 1.7384 - acc: 0.1875 - val_loss: 1.6850 - val_acc: 0.2500
Epoch 2/5
15/15 [==============================] - 56s 4s/step - loss: 1.6441 - acc: 0.2292 - val_loss: 1.8458 - val_acc: 0.2250
Epoch 3/5
15/15 [==============================] - 57s 4s/step - loss: 1.6262 - acc: 0.3042 - val_loss: 1.6137 - val_acc: 0.3625
Epoch 4/5
15/15 [==============================] - 55s 4s/step - loss: 1.6000 - acc: 0.3417 - val_loss: 1.4458 - val_acc: 0.2750
Epoch 5/5
15/15 [==============================] - 60s 4s/step - loss: 1.5329 - acc: 0.3292 - val_loss: 1.4682 - val_acc: 0.3000


In [ ]:
"""==========
   모델 학습하기_2
   =========="""
model.fit_generator(train_generator, epochs=3, validation_data=validation_generator)

Epoch 1/3
279/279 [==============================] - 979s 4s/step - loss: 1.3817 - acc: 0.4188 - val_loss: 1.2299 - val_acc: 0.4420
Epoch 2/3
 10/279 [>.............................] - ETA: 19:19 - loss: 1.2601 - acc: 0.4812

In [ ]:
"""==========
   모델 평가하기
   =========="""

print("-- Evaluate --")
scores = model.evaluate_generator(validation_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
# 모델 저장
model_json = model.to_json()
with open("model/model.json", "w") as json_file : 
    json_file.write(model_json)

# 모델 weight 저장
keras_file="weight/garbage.h5"
model.save(keras_file)